# ARG Jupyter Notebook

*Find antibiotic resistance genes in metagenomic sequences 🧬*

## TOC:
* Set-Up
* Obtain metagenomic sequences
* Obtain antibiotic resistance gene sequences
* Create blast database
* Run blast of sequences against database
* Organize results into table

##  Set-Up BLAST:
Please refer to https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/ to download the best version for your machine. This example will use a machine running Linux. 🐧

In [ ]:
#Download and Install Blast
!wget -q https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/ncbi-blast-2.13.0+-x64-linux.tar.gz
!tar -xf ncbi-blast-2.13.0+-x64-linux.tar.gz

In [ ]:
#Check to see if running
!./ncbi-blast-2.13.0+/bin/blastp -h

##  Set-Up rush:
Rush is "a cross-platform command-line tool for executing jobs in parallel" that will be used throughout the experiment. Rush's github can be found here: https://github.com/shenwei356/rush (Still using Linux download, change if needed.)

In [ ]:
#Download and Install rush
!wget -q https://github.com/shenwei356/rush/releases/download/v0.4.2/rush_linux_amd64.tar.gz
!tar -xf rush_linux_amd64.tar.gz

In [ ]:
#Check to see if running
!./rush -h

## Set-Up seqtk and Obtain Metagenomic Sequences
If files are fastq, convert to fasta using seqtk, "a fast and lightweight tool for processing sequences in the FASTA or FASTQ format." Seqtk's github can be found here: https://github.com/lh3/seqtk

In [ ]:
#Clone and make seqtk
!git clone https://github.com/lh3/seqtk.git;
!cd seqtk; make

In [ ]:
#Check to see if running
!./seqtk/seqtk

In [ ]:
#Place all fastq files into one directory for example: fastq_files_in
#Create directory for out files. For example: fasta_files_out
#Convert fastq to fasta using rush to parallelize
!cd fastq_files_in && ls * | ../rush "../seqtk/seqtk seq -a {} > ../fasta_files_out/{..}.fasta"

## Obtain Antibiotic Resistance Gene Sequences
Go to Comprehensive Antibiotic Resistance Database (CARD) and download antibiotic inactivation genes:<br>
https://card.mcmaster.ca/download?id=36000&name=antibiotic%20inactivation<br><br>
Select:
* *part_of*
* *is_a*
* *participates_in*
* *has_part*
* *Nucleotide*

In [ ]:
#Create ARG_seq directory and put in nucleotide files from CARD after unzipping
!mkdir ARG_seq
#Create db directory in which to create directory
!mkdir db

## Create blast db with sequences from CARD

In [ ]:
#Use name of fasta file downloaded from CARD
!ncbi-blast-2.13.0+/bin/makeblastdb -in ARG_seq/ARG_nucleotide.fasta -dbtype 'nucl' -out db/ARG 

## Run BLAST against database

In [ ]:
#E-value
evalue = "-evalue 10"
#100% Identity Results Only
identity = "-perc_identity 100"
#100% Query Coverage
coverage = "-qcov_hsp_perc 100"
cmd = ('ls * | ../rush "../ncbi-blast-2.13.0+/bin/blastn -query {} '+
       evalue+" "+identity+" "+coverage+
       ' -db ../db/ARG -out ../blast_out/{.}_blast.txt -outfmt 10"')
!cd fasta_files_out && {cmd}

## Organize Results into Table

In [12]:
import os
import glob
import pandas as pd
os.chdir("blast_out")
files = glob.glob("*_blast.txt")
big_df = pd.DataFrame()
df_dict = {}
for file in files:
    df = pd.read_csv(file, sep=',', header=None)
    df.columns = ["query_acc.ver", "subject_acc.ver", "%_identity", "alignment_length", "mismatches", "gap_opens", "q._start", "q._end", "s._start", "s._end", "evalue", "bit_score"]
    name = str("ARG_hits_data/"+file[:-4]+".csv")
    filename = file[:-10]
    df.to_csv(name)
    df[filename] = pd.Series([1 for x in range(len(df.index))])
    new_df = df[["subject_acc.ver",filename]]
    counted_df = new_df.groupby(by=["subject_acc.ver"]).sum()
    big_df = pd.concat([big_df, counted_df], axis=1)
    big_df = big_df.fillna(0)
    sorted_df = counted_df.sort_values(by=filename, ascending=False)
    df_dict.update({filename:df["subject_acc.ver"]})
df_2=pd.DataFrame.from_dict(df_dict,orient='index').transpose()
big_df.to_csv("ARG_hits.csv")
df_2.to_csv("ARG_list.csv")
os.chdir("../")